In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
HISTORY = 6
FUTURE = 1
OVERLAP = 3

In [3]:
data = pd.read_csv("C:/Users/ay0un/Desktop/ds-project/buenos-aires/Air-Quality-Forecast-Challenge/1h_sampled_data.csv")

In [4]:
data.columns

Index(['time', 'temp_pi', 'hum_pi', 'PM1_pi', 'PM2.5_pi', 'PM10_pi',
       'temp_blue', 'precip', 'hum_blue', 'wind_speed', 'wind_direction',
       'cloud_cover', 'date', 'temp_sbs', 'hum_sbs', 'wind', 'pressure',
       'PM2.5', 'PM10', 'AQI', 'sunrise', 'noon', 'sunset', 'daylight'],
      dtype='object')

In [5]:
data['time'] = pd.to_datetime(data['time'], utc=True)

In [6]:
data['time']

0     2021-03-23 09:00:00+00:00
1     2021-03-23 10:00:00+00:00
2     2021-03-23 11:00:00+00:00
3     2021-03-23 12:00:00+00:00
4     2021-03-23 13:00:00+00:00
                 ...           
480   2021-04-12 08:00:00+00:00
481   2021-04-12 09:00:00+00:00
482   2021-04-12 10:00:00+00:00
483   2021-04-12 11:00:00+00:00
484   2021-04-12 12:00:00+00:00
Name: time, Length: 485, dtype: datetime64[ns, UTC]

In [7]:
hours = data['time'].dt.hour

In [8]:
data['hour'] = hours

In [9]:
data.head()

,time,temp_pi,hum_pi,PM1_pi,PM2.5_pi,PM10_pi,temp_blue,precip,hum_blue,wind_speed,...,wind,pressure,PM2.5,PM10,AQI,sunrise,noon,sunset,daylight,hour
0,2021-03-23 09:00:00+00:00,8.316667,53.283333,6.042500,11.798333,17.710000,3.106314,0.0,76.0,5.116561,...,3.7,1025.0,16.0,24.3,59.0,2021-03-23 06:22:52.709917+01:00,2021-03-23 12:32:27+01:00,2021-03-23 18:42:36.960331+01:00,0 days 12:19:44.250414,9
1,2021-03-23 10:00:00+00:00,13.408333,38.358333,4.840833,10.010000,15.388333,5.416314,0.0,67.0,6.924738,...,3.7,1025.0,14.0,20.9,55.0,2021-03-23 06:22:52.709917+01:00,2021-03-23 12:32:27+01:00,2021-03-23 18:42:36.960331+01:00,0 days 12:19:44.250414,10
2,2021-03-23 11:00:00+00:00,17.391667,32.191667,5.140833,7.510833,9.632500,7.216313,0.0,55.0,7.421590,...,14.8,1025.0,12.0,17.3,50.0,2021-03-23 06:22:52.709917+01:00,2021-03-23 12:32:27+01:00,2021-03-23 18:42:36.960331+01:00,0 days 12:19:44.250414,11
3,2021-03-23 12:00:00+00:00,18.650000,30.083333,3.752500,6.064167,8.284167,8.656314,0.0,48.0,6.952755,...,14.8,1025.0,NaN,NaN,27.0,2021-03-23 06:22:52.709917+01:00,2021-03-23 12:32:27+01:00,2021-03-23 18:42:36.960331+01:00,0 days 12:19:44.250414,12
4,2021-03-23 13:00:00+00:00,19.983333,28.125000,4.610000,10.042500,15.750000,9.586313,0.0,39.0,11.269782,...,11.1,1025.0,11.0,16.2,45.0,2021-03-23 06:22:52.709917+01:00,2021-03-23 12:32:27+01:00,2021-03-23 18:42:36.960331+01:00,0 days 12:19:44.250414,13


In [10]:
features = ['hour','temp_pi', 'hum_pi', 'wind_speed','precip']
targets = ['PM2.5_pi', 'PM10_pi'] # raspberry-pi measurement
targets2= ['PM2.5', 'PM10'] # From external source

In [11]:
pipi = ['hour','temp_pi', 'hum_pi', 'wind_speed','precip', 'PM2.5_pi', 'PM10_pi'] 
pmpm = ['hour','temp_pi', 'hum_pi', 'wind_speed','precip', 'PM2.5', 'PM10']

In [12]:
data_pi = data[pipi]

In [13]:
data_pm = data[pmpm]

In [14]:
index = data_pi['temp_pi'].index[data['temp_pi'].apply(np.isnan)]
index

Int64Index([204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,
            217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229,
            230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242,
            419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431,
            432, 433],
           dtype='int64')

In [15]:
len(index)

54

In [16]:
data_pi = data_pi.dropna()
data_pm = data_pm.dropna()

In [17]:
data_pi

,hour,temp_pi,hum_pi,wind_speed,precip,PM2.5_pi,PM10_pi
0,9,8.316667,53.283333,5.116561,0.0,11.798333,17.710000
1,10,13.408333,38.358333,6.924738,0.0,10.010000,15.388333
2,11,17.391667,32.191667,7.421590,0.0,7.510833,9.632500
3,12,18.650000,30.083333,6.952755,0.0,6.064167,8.284167
4,13,19.983333,28.125000,11.269782,0.0,10.042500,15.750000
...,...,...,...,...,...,...,...
480,8,5.350000,77.241667,16.935310,1.2,6.740000,10.450833
481,9,5.975000,75.625000,17.917007,0.9,3.617500,5.262500
482,10,6.372727,72.300000,13.684735,0.8,8.692727,13.842727
483,11,6.641667,71.333333,13.608762,0.4,9.065833,13.411667


In [21]:
pipi_new = []
features_new = []
for i in range(HISTORY):
    pipi_new += [f + "_t" + str(i) for f in pipi]

In [22]:
features_new += pipi_new + targets

In [23]:
pmpm_new = []
for i in range(HISTORY):
    pmpm_new += [f + "_t" + str(i) for f in pmpm]

In [24]:
features_new_pm = []
features_new_pm += pmpm_new + targets

In [25]:
x = math.ceil(data_pi.shape[0]/OVERLAP)
y = len(pipi)*HISTORY + len(targets) # 7 inputs (of 6 hours) + 7th hour output
pi_new = np.zeros((x, y))
pm_new = np.zeros((x, y))

In [26]:
pi_new.shape

(144, 44)

In [27]:
pi_new.shape # 6 x 7 + 2 

(144, 44)

In [52]:
len(data_pi)/6*2

143.66666666666666

In [29]:
# Raspberry pi
k = 0
for i in range(HISTORY, data_pi.shape[0], OVERLAP):
    piece = data_pi.loc[i-HISTORY:i-1, :]

    if len(piece) == 0: # check missing value
        print(i)
    
    for j in range(len(piece)):
        pi_new[k, j*(len(piece.columns)):(j+1)*len(piece.columns)] = piece.iloc[j,:]
        #print(pi_new)
    pi_new[k, -2] = data_pi.iloc[i, -2] # output PM2.5
    pi_new[k, -1] = data_pi.iloc[i, -1] # output PM10
    k += 1

210
213
216
219
222
225
228
231
234
237
240
243
426
429


In [77]:
data_pi.loc[200-6:220, : ]

,hour,temp_pi,hum_pi,wind_speed,precip,PM2.5_pi,PM10_pi
194,10,20.525000,39.050000,8.089993,0.0,7.592500,11.110000
195,11,26.716667,31.075000,7.952660,0.0,9.481667,12.852500
196,12,29.566667,27.775000,6.287130,0.0,8.589167,12.651667
197,13,30.691667,25.600000,6.120000,0.0,11.270000,18.164167
198,14,30.308333,25.808333,5.447788,0.0,6.501667,9.085000
199,15,27.133333,28.058333,5.040000,0.0,9.916667,15.684167
200,16,23.441667,33.358333,4.334974,0.0,10.040000,14.133333
201,17,22.225000,35.925000,5.191994,0.0,10.822500,14.894167
202,18,20.666667,38.791667,9.164103,0.0,13.584167,20.088333
203,19,19.080000,44.990000,7.568566,0.0,10.814000,15.249000


In [30]:
time_series_pi = pd.DataFrame(pi_new, columns= features_new )

In [31]:
time_series_pi

,hour_t0,temp_pi_t0,hum_pi_t0,wind_speed_t0,precip_t0,PM2.5_pi_t0,PM10_pi_t0,hour_t1,temp_pi_t1,hum_pi_t1,...,PM10_pi_t4,hour_t5,temp_pi_t5,hum_pi_t5,wind_speed_t5,precip_t5,PM2.5_pi_t5,PM10_pi_t5,PM2.5_pi,PM10_pi
0,9.0,8.316667,53.283333,5.116561,0.0,11.798333,17.710000,10.0,13.408333,38.358333,...,15.750000,14.0,17.625000,30.933333,9.826088,0.0,9.652500,15.223333,6.641000,9.477000
1,12.0,18.650000,30.083333,6.952755,0.0,6.064167,8.284167,13.0,19.983333,28.125000,...,16.762500,17.0,11.616667,40.350000,7.754637,0.0,6.040000,9.553333,7.147500,10.535000
2,15.0,13.990000,35.770000,8.209263,0.0,6.641000,9.477000,16.0,12.683333,37.733333,...,12.093333,20.0,8.466667,49.616667,7.895416,0.0,8.685833,12.543333,8.777500,13.442500
3,18.0,10.425000,42.900000,6.162207,0.0,7.147500,10.535000,19.0,9.416667,45.208333,...,12.672500,23.0,6.516667,61.908333,5.692100,0.0,9.979167,14.669167,7.956667,10.690833
4,21.0,7.566667,53.983333,6.849467,0.0,8.777500,13.442500,22.0,6.850000,57.158333,...,19.001667,2.0,4.981818,66.109091,4.510787,0.0,12.583636,18.430000,13.833333,20.941667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,17.0,16.958333,33.033333,19.130875,0.0,6.969167,10.899167,18.0,16.300000,31.950000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,3.560000,4.923000
140,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,6.740000,10.450833
141,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,9.065833,13.411667
142,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [35]:
# Remove missing data

time_series_pi = time_series_pi.iloc[:138, :]
time_series_pi

,hour_t0,temp_pi_t0,hum_pi_t0,wind_speed_t0,precip_t0,PM2.5_pi_t0,PM10_pi_t0,hour_t1,temp_pi_t1,hum_pi_t1,...,PM10_pi_t4,hour_t5,temp_pi_t5,hum_pi_t5,wind_speed_t5,precip_t5,PM2.5_pi_t5,PM10_pi_t5,PM2.5_pi,PM10_pi
0,9.0,8.316667,53.283333,5.116561,0.0,11.798333,17.710000,10.0,13.408333,38.358333,...,15.750000,14.0,17.625000,30.933333,9.826088,0.0,9.652500,15.223333,6.641000,9.477000
1,12.0,18.650000,30.083333,6.952755,0.0,6.064167,8.284167,13.0,19.983333,28.125000,...,16.762500,17.0,11.616667,40.350000,7.754637,0.0,6.040000,9.553333,7.147500,10.535000
2,15.0,13.990000,35.770000,8.209263,0.0,6.641000,9.477000,16.0,12.683333,37.733333,...,12.093333,20.0,8.466667,49.616667,7.895416,0.0,8.685833,12.543333,8.777500,13.442500
3,18.0,10.425000,42.900000,6.162207,0.0,7.147500,10.535000,19.0,9.416667,45.208333,...,12.672500,23.0,6.516667,61.908333,5.692100,0.0,9.979167,14.669167,7.956667,10.690833
4,21.0,7.566667,53.983333,6.849467,0.0,8.777500,13.442500,22.0,6.850000,57.158333,...,19.001667,2.0,4.981818,66.109091,4.510787,0.0,12.583636,18.430000,13.833333,20.941667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,23.0,6.787500,58.587500,7.636753,0.0,8.486250,14.362500,0.0,6.350000,59.500000,...,6.323636,4.0,4.983333,61.616667,4.735060,0.0,7.600000,11.365833,5.475556,8.303333
134,2.0,5.500000,61.180000,5.860375,0.0,5.630000,9.211000,3.0,5.545455,59.136364,...,12.845833,7.0,6.908333,55.350000,4.024922,0.0,7.712500,10.448333,4.786250,7.621250
135,5.0,4.850000,62.908333,4.680000,0.0,8.113333,12.313333,6.0,5.850000,59.925000,...,14.958333,10.0,15.133333,30.958333,9.339208,0.0,5.928333,8.614167,5.369000,8.341000
136,8.0,8.858333,44.375000,5.191994,0.0,8.003333,13.353333,9.0,11.708333,42.016667,...,12.346667,13.0,22.591667,21.891667,14.400000,0.0,8.453333,14.123333,3.553750,5.736250


In [36]:
time_series_pi.to_csv('time_series_pi.csv')